In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import random
import tqdm
import copy
from chemprop.args import TrainArgs, PredictArgs
from chemprop.train import cross_validate, run_training, make_predictions
from chemprop.data import get_data
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score

In [4]:
wandb.init(project='moleculenet10', entity='chrisxx')
config = wandb.config
config.dataset = 'clintox' # supported 'bbbp', 'clintox', 'hiv', 'tox21'
config.model_dir = '../../models/moleculenet/chemprop_mpnn/%s/'%config.dataset
config.data_dir = '../../datasets/mymoleculenet/%s/'%config.dataset
config.seed = 123

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-08-26 15:10:51.958128: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt

In [5]:
random.seed(config.seed)
torch.manual_seed(config.seed)
np.random.seed(config.seed)

In [ ]:
rocs = []
prcs = []
for rep in range(10):
    train_args = [
        '--data_path', '%s%d/train.csv'%(config.data_dir, rep),
        '--separate_val_path', '%s%d/valid.csv'%(config.data_dir, rep),
        '--separate_test_path', '%s%d/test.csv'%(config.data_dir, rep),
        '--dataset_type', 'classification',
        '--save_dir', '%s%d/'%(config.model_dir, rep),
        '--num_folds', '1',
        '--smiles_columns', 'smiles',
        '--target_columns', 'target',
        #'--no_cuda'
    ]
    
    os.makedirs("%s%d"%(config.model_dir, rep), exist_ok=True)
    score = cross_validate(args=TrainArgs().parse_args(train_args), train_func=run_training)
    
    test_args = [
            '--test_path', '%s%d/test.csv'%(config.data_dir, rep),
            '--preds_path', '%s%d/pred.csv'%(config.model_dir, rep),
            '--checkpoint_dir', '%s%d/'%(config.model_dir, rep),
            '--num_workers', '0',
            '--smiles_columns', 'smiles',
            #'--no_cuda'
        ]
    preds = make_predictions(args=PredictArgs().parse_args(test_args))
    preds = np.asarray(preds)[:, 0]
    gt = pd.read_csv('%s%d/test.csv'%(config.data_dir, rep))
    roc = roc_auc_score(gt['target'].to_numpy(), preds)
    prc = average_precision_score(gt['target'].to_numpy(), preds)
    print(roc, prc)
    wandb.log({'roc_test_avg20':roc, 'prc_test_avg20':prc}, step=25*rep)
    rocs += [roc]
    prcs += [prc]
wandb.log({'roc_test_avgavg': np.mean(rocs),
           'roc_test_avgstd': np.std(rocs),
           'prc_test_avgavg': np.mean(prcs),
           'prc_test_avgstd': np.std(prcs)})
wandb.run.summary["roc_test_mean"] = np.mean(rocs) 
wandb.run.summary["roc_test_std"] = np.std(rocs)
wandb.run.summary["prc_test_mean"] = np.mean(prcs)
wandb.run.summary["prc_test_std"] = np.std(prcs)    

Command line
python /home/chrisw/.cache/pypoetry/virtualenvs/graph-transformer-9jPERXQ--py3.8/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/chrisw/.local/share/jupyter/runtime/kernel-1fa8ea27-29cb-4009-aabc-8044eb62c3bf.json
Args
{'activation': 'ReLU',
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_dir': None,
 'crossval_index_file': None,
 'crossval_index_sets': None,
 'cuda': True,
 'data_path': '../../datasets/mymoleculenet/clintox/0/train.csv',
 'data_weights_path': None,
 'dataset_type': 

 96%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊       | 23/24 [00:02<00:00, 11.42it/s]
                                                                                                                                                                                                                   
 67%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                          | 2/3 [00:00<00:00, 14.35it/s]
                                                                                                                                                                                                                   Validation auc = 0.543590
  3%|█████▊                                                                                                                    

                                                                                                                                                                                                                   
  0%|                                                                                                                                                                                        | 0/3 [00:00<?, ?it/s]
                                                                                                                                                                                                                   Validation auc = 0.716239
 13%|███████████████████████▎                                                                                                                                                       | 4/30 [00:04<00:27,  1.07s/it]Epoch 4

 12%|█████████████████████▉                                                                                            

                                                                                                                                                                                                                   Validation auc = 0.825071
 23%|████████████████████████████████████████▊                                                                                                                                      | 7/30 [00:07<00:21,  1.07it/s]Epoch 7

 17%|█████████████████████████████▏                                                                                                                                                 | 4/24 [00:00<00:00, 37.29it/s]Loss = 1.9755e-01, PNorm = 34.8126, GNorm = 0.3905, lr_0 = 6.3276e-04

 50%|███████████████████████████████████████████████████████████████████████████████████████                                                                                       | 12/24 [00:00<00:00, 32.09it/s]Loss = 1.7116e-01, PNorm = 34.8529, GNorm = 0.2463

  0%|                                                                                                                                                                                       | 0/24 [00:00<?, ?it/s]Loss = 1.6899e-01, PNorm = 35.1258, GNorm = 0.2265, lr_0 = 4.9090e-04

 50%|███████████████████████████████████████████████████████████████████████████████████████                                                                                       | 12/24 [00:00<00:00, 35.20it/s]Loss = 1.7391e-01, PNorm = 35.1770, GNorm = 0.2059, lr_0 = 4.7366e-04

 83%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                             | 20/24 [00:00<00:00, 31.67it/s]Loss = 1.5545e-01, PNorm = 35.2213, GNorm = 0.3017, lr_0 = 4.5702e-04

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 34.74it/s]
                                                                                                                                                                                                                   
  0%|                                                                                                                                                                                        | 0/3 [00:00<?, ?it/s]
                                                                                                                                                                                                                   Validation auc = 0.829044
 27%|██████████████████████████████████████████████▋                                                                           

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 33.50it/s]
                                                                                                                                                                                                                   
  0%|                                                                                                                                                                                        | 0/3 [00:00<?, ?it/s]
                                                                                                                                                                                                                   Validation auc = 0.848652
 37%|███████████████████████████████████████████████████████████████▊                                                          

  0%|                                                                                                                                                                                        | 0/3 [00:00<?, ?it/s]
                                                                                                                                                                                                                   Validation auc = 0.865809
 47%|█████████████████████████████████████████████████████████████████████████████████▏                                                                                            | 14/30 [00:12<00:14,  1.14it/s]Epoch 14

 33%|██████████████████████████████████████████████████████████▎                                                                                                                    | 8/24 [00:00<00:00, 31.20it/s]Loss = 1.6566e-01, PNorm = 35.4809, GNorm = 0.2069, lr_0 = 3.4089e-04

 67%|███████████████████████████████████████████

 57%|██████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                           | 17/30 [00:15<00:11,  1.12it/s]Epoch 17

 33%|██████████████████████████████████████████████████████████▎                                                                                                                    | 8/24 [00:00<00:00, 30.91it/s]Loss = 1.4202e-01, PNorm = 35.7200, GNorm = 0.7856, lr_0 = 2.6446e-04

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                          | 16/24 [00:00<00:00, 32.68it/s]Loss = 1.3719e-01, PNorm = 35.7503, GNorm = 1.0827, lr_0 = 2.5517e-04

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 33.78it/s]
   

 17%|█████████████████████████████▏                                                                                                                                                 | 4/24 [00:00<00:00, 37.04it/s]Loss = 1.1297e-01, PNorm = 35.9084, GNorm = 0.8918, lr_0 = 2.0517e-04

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                          | 16/24 [00:00<00:00, 31.89it/s]Loss = 1.1762e-01, PNorm = 35.9315, GNorm = 1.0413, lr_0 = 1.9796e-04

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 33.51it/s]
                                                                                                                                                                                                                   
  0%|       